In [1]:
import pandas as pd
from json import load

df_geo = pd.read_csv('./data/geolocalizacion.csv', sep=';')
df_car = pd.read_csv('./data/caracteristicas.csv', sep=';')
df_web = pd.read_csv('./data/web.csv', sep=';')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
"""
Desechamos las columnas de metadatos que no van a influir en nuestro knn. 
Esta información, en el caso de llegar a representarla lo que haremos será enriquecer con una api en una tabla exclusiva para ello 
ennuestra BBD


necesario:  
            [rk] necesitamos el identificador del piso.
            [planta] es un valor muy necesario para nuestro  algoritmo

útilies:   
            [geo] no es necesario si los barrios estuviesen bien definidios. pero como aún no tenemos clara la manera de adquirir el barrio lo mantenemos.
            [CP] si viéramos que los barrios son una lista de codigos postales podríamos ahorrarnos el uso del shapefile y simplemente relacionar CP con barrios. 
            [Pais] provincia y ciudad podrían ser útiles si están bien metidos los datos.
            [distrito y barrio] están un poco sucios pero los mantenemmos aunque luego lo machaquemos con nuestraa información.df_geo
            [calle] podría no ser necesario para nuestro algoritmo pero por ahora lo mantenemos, seguramente lo desechemos más adelante.
"""
df_meta_geo = df_geo[['rk', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia', 'ciudad', 'distrito', 'barrio', 'calle', 'planta']]


In [3]:

"""
Desechamos las columnas de datos que no crremos que nos vayan a influir.complex

eliminamos: 
            [rk] no necesitamos saber en una misma columna el estado del piso y su identificador. lo tenemos cada uno en su columna correspondiente. Posteriormente filtraremos aquellos pisos repetidos que por haber sido reformados nos siguen apareciendo sin reformar.Además tendremos que ser consistenetes con la nomenclartura de esta columna ya que rk de la tabla de geo y uv de esta tabla contienen la misma información, no es así con rk de esta tabla.complex

            [metrica] es inconsistente que tengamos una columna "metros" y redundemos en que la métrica es m2. Asumimos que todos los inmuebles están en metros y que si fuese necesario incluir en medidas imperiales (por ejemplo) se transformaría a metros o se transformaría a metros y se incluría una columna con la información en yardas en todas las viviendas.complex

            [certificaciones] [certificado_energetico] [seguro_hogar]
            

mantenemos: 
            Muy importante para nuestro algoritmo
            [ascensor ]

            Podría ser interesante para nuestro algoritmo 
            [fecha_construccion] [terraza] [orientacion] [situacion][portal][parque_infantil] [trastero] [piscina_comunitaria] [piscina] [cuarto_de_basura] [zonas_comunes] [garaje] [tejado] [calefaccion]  [jardin] [aire_acondicionado] [seguro_hogar]
            

Mantenemos muchas columnas poco relevantes pero el poderador les dará su valor.

NOTA: columna  [portal] es si tiene o no portero automático. columana [techo] es si está o no abuardillado 

"""
df_meta_car=df_car[['uv', 'estado', 'habitaciones', 'banos','metros','fecha_construccion','orientacion','situacion', 'portal', 'ascensor', 'parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura','zonas_comunes', 'piscina', 'garaje', 'tejado', 'calefaccion', 'jardin', 'aire_acondicionado']]


In [4]:

"""
por qué sale? /home/emilio/.local/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
"""
def limpia_estado(x):
    if x != '2' and x !='1': return '3'
    else: return x

df_meta_car.estado=df_meta_car.estado.apply(limpia_estado).copy()


/home/emilio/.local/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
"""
Mantenemos prácticamente todos los datos.

Alunos comentarios:
    [estado] debería ser redundante ya que lo tenemos tb en df_meta_car pero nos servirá para asegurarnos de que el dato es correcto
    [precio_antiguo] [descuento] los desechamos, nuestro algoritmo sólo considerará el precio actual. si es con descuento o no será el comercial quien tenga esta información y recalcará al clliente que se estaría aprovechando de un descuento.
    [gastos_comunitarios] esta variable podría ser interesante para nuestro algoritmo así que la mantenemos. en el momento de recomendar una u otra casa puede que sí sea información valiosa para los comerciales: nos convendría vender antes un piso con una cuota de gastos comunitarios alta que una con pocos gastos comunitarios.
    
    eliminanos
    [moneda] suponemos todos los precios en eurosç
    [estado] nos quedamos con el de la tabla características
"""
df_meta_web=df_web[['uv', 'tipo_vivienda', 'precio', 'gastos_comunitarios']]


In [6]:
df_meta_geo.columns=['uv', 'latitud', 'longitud', 'codigo_postal', 'pais', 'provincia',
       'ciudad', 'distrito', 'barrio', 'calle', 'planta']
#mantenemmos consistente nuestra nomenclatura del identificador y llamamos a todos los uv correctamente (no rk)


In [7]:
def saca_NS(cadena):
    CADENA=cadena.upper()
    if 'NOR' in CADENA: return 'N'
    elif 'SU' in CADENA: return 'S'
    else: return ''

def saca_EO(cadena):
    CADENA=cadena.upper()
    if 'OES' in CADENA: return 'O'
    elif 'EST' in CADENA: return 'E'
    else: return ''

In [8]:
df_meta_car['NS']=df_meta_car.orientacion
df_meta_car['EO']=df_meta_car.orientacion

df_meta_car['NS']=df_meta_car['orientacion'].apply(saca_NS)
df_meta_car['EO']=df_meta_car['orientacion'].apply(saca_EO)
df_meta_car.drop(['orientacion'], axis=1, inplace= True)

<ipython-input-8-62264272fed9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car['NS']=df_meta_car.orientacion
<ipython-input-8-62264272fed9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_car['EO']=df_meta_car.orientacion
<ipython-input-8-62264272fed9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [9]:
#pasamos todos los valores a enteros
df_meta_car[[ 'estado', 'habitaciones', 'banos', 'metros','fecha_construccion', 'tejado']] = df_meta_car[[ 'estado', 'habitaciones', 'banos', 'metros','fecha_construccion', 'tejado']].fillna(0.0).astype(int)

df_meta_car.situacion=df_meta_car.situacion.fillna('NO').map({'Exterior':int(3),"Patio de manzana":int(2),"Interior":int(1), 'NO':int(0)}).astype(int)

df_meta_car[['portal', 'ascensor','parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina', 'garaje', 'tejado','calefaccion','jardin', 'aire_acondicionado']] = df_meta_car[['portal', 'ascensor','parque_infantil', 'terraza', 'trastero', 'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina', 'garaje', 'tejado','calefaccion','jardin', 'aire_acondicionado']].fillna(0).astype(int)


/home/emilio/.local/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
def texto_mas_frecuente(x):
    if x.empty or x.isnull().all():
        return ''
    else: 
        x = x.dropna()
        return x.mode()


df_meta_car = df_meta_car.groupby('uv', as_index=False).agg(
    {
    'estado': 'max',
    'habitaciones': 'max',
    'banos': 'max',
    'metros': 'max',
    'fecha_construccion': 'max',
    'NS': texto_mas_frecuente,
    'EO': texto_mas_frecuente,
    'situacion': 'max',
    'portal': 'max',
    'ascensor': 'max',
    'parque_infantil': 'max',
    'terraza': 'max',
    'trastero': 'max',
    'piscina_comunitaria': 'max',
    'cuarto_de_basura': 'max',
    'zonas_comunes': 'max',
    'piscina': 'max',
    'garaje': 'max',
    'tejado': 'max',
    'calefaccion': 'max',
    'jardin': 'max',
    'aire_acondicionado': 'max',
    }
)     


In [11]:
df_meta_web = df_meta_web.groupby('uv', as_index=False).agg(
    {
    'tipo_vivienda': texto_mas_frecuente,
    'precio': 'max',
    'gastos_comunitarios': 'max',
    }
)     



In [12]:
df_meta_geo = df_meta_geo.groupby('uv', as_index=False).agg(
    {
    'latitud': 'max',
    'longitud': 'max',
    'codigo_postal': 'max',
    'pais': 'max',
    'ciudad': texto_mas_frecuente,
    'distrito': texto_mas_frecuente,
    'calle': texto_mas_frecuente,
    'barrio': texto_mas_frecuente,
    'planta': 'max',
    }
)     



In [13]:

def filtrar_tipo_vivienda(cadena):
    """"
    Esta función abre  el archivo "./config/opciones.json" y obtiene el tipo de viviendas posibles en nuestra base de datos.
    Si no está en esa lista devuelve el valor "OTRO"
    Arg: cadena de tipo string

    Return: string conteniendo el tipo de vivienda permitida
    """
    try:
        with open("./config/opciones.json") as archivo:
            opciones = load(archivo)
        conjunto_vivientas = set(opciones['conjunto_vivientas'])
    except :
            raise("Error leyendo 'opciones.json'")

    CADENA = cadena.upper()
    if CADENA in conjunto_vivientas: return CADENA
    elif CADENA == 'MORAIDA': return 'CASA' #alguna casa del dataset venía con la palabra MORAIDA y parece un error hemos decidido hardcodear por ser algo concreto.
    else: return 'OTRO'


In [14]:
#limpiamos el df fijando tipo de vivienda, poniendo el precio de la vivienda en miles de euros K, pasando estado a 1 sin reformar, 2 reformado, 3 vendido. usamos int
df_meta_web.tipo_vivienda = df_meta_web.tipo_vivienda.apply(filtrar_tipo_vivienda)
df_meta_web.precio =  df_meta_web.precio.fillna(0).astype(int)
df_meta_web.precio =  (df_meta_web.precio/1000).astype(int)
df_meta_web.gastos_comunitarios =  df_meta_web.gastos_comunitarios.fillna(0).astype(int)
#renombramos las nuevas variables
df_meta_web.columns=['uv', 'tipo_vivienda', 'precio_k', 'gastos_comunitarios']
#las coordenadas vienen multiplicadas por 100
#2do incluit este ajuste en el json
df_meta_geo[['latitud', 'longitud']]=df_meta_geo[['latitud', 'longitud']]/100

In [15]:
df_meta_web.set_index('uv', inplace=True)
df_meta_geo.set_index('uv', inplace=True)
df_meta_car.set_index('uv', inplace=True)


In [19]:
df=df_meta_car.join(df_meta_geo, how='outer').join(df_meta_web, how='outer')

In [26]:
df

,estado,habitaciones,banos,metros,fecha_construccion,NS,EO,situacion,portal,ascensor,...,codigo_postal,pais,ciudad,distrito,calle,barrio,planta,tipo_vivienda,precio_k,gastos_comunitarios
uv,,,,,,,,,,,,,,,,,,,,,
UV/2020/000001,1,2,1,53,1963,N,,3,0,1,...,28019,1,Madrid,Carabanchel,Avenida de Oporto,,1,NaN,NaN,NaN
UV/2020/000003,1,2,1,45,1960,N,,3,0,1,...,28037,1,Madrid,San Blas,Avenida Institucion Libre de Enseñanza,Canillejas,9,PISO,160.0,0.0
UV/2020/000004,1,2,1,67,1966,,O,1,0,1,...,28341,1,Valdemoro,Zona estación,Calle Tenerife,,1,PISO,125.0,0.0
UV/2020/000005,1,3,2,101,1972,S,O,3,0,1,...,28024,1,Madrid,Latina,Calle Cine,,2,NaN,NaN,NaN
UV/2020/000006,1,2,1,62,1960,S,,3,0,0,...,28037,1,Madrid,Ciudad Lineal,Avenida Institución Libre de Enseñanza,,2,PISO,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UV/2021/000448,1,2,1,56,1970,S,O,1,0,0,...,28017,1,Madrid,Ciudad Lineal,Emilio Ferrari,,3,PISO,130.0,30.0
UV/2021/000452,1,2,1,118,2004,,O,3,0,1,...,28027,1,Madrid,La peseta,test,Carabanchel,1,CASA,0.0,12.0
UV/2021/000455,1,0,0,40,1900,N,E,1,0,0,...,28027,1,Madrid,La latina,TEST,Aluche,1,CASA,100.0,1.0
